In [1]:
from utils.data_handling import *
from utils.sensor_optimisation_gpy_sparse import *
from parameters import *

import matplotlib.pyplot as plt

np.random.seed(101)

In [2]:
parameters['i_end'] = 100
parameters

{'i_start': 0,
 'i_end': 100,
 'crop': ((-10, 10), (-10, 10), (0, 50)),
 'field_name': 'TracerBackground',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

  0%|          | 0/1 [00:00<?, ?it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0
==> Cropping vtu files to ((-10, 10), (-10, 10), (0, 50))


100%|██████████| 1/1 [00:00<00:00,  2.32it/s]

Number of Locations after cropping :  1129
==> Loading from : ../data/temp_data/cache_0_100_((-10, 10), (-10, 10), (0, 50))/loc_TracerBackground.pkl
==> Loading from : ../data/temp_data/cache_0_100_((-10, 10), (-10, 10), (0, 50))/time_TracerBackground.pkl
==> Loading from : ../data/temp_data/cache_0_100_((-10, 10), (-10, 10), (0, 50))/data_TracerBackground.pkl


In [4]:
# Data for the regression
dim = 3
t = 100
X = loc_df.values #[I,:dim]
Y = data_df.values[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Y.shape

(1129, 1)

Setting the kernel

In [5]:
kernel = GPy.kern.Matern52(dim,ARD=False) + GPy.kern.White(dim)
model_gp = GPy.models.GPRegression(X,Y,kernel)
#model_gp.optimize(messages=True, max_iters = 1000)


## Sensor Optimisation

Define the Sets for the optimisation

In [6]:
n_V = X.shape[0]
sets = define_sets(n_V)

Number of sensors to place : 

In [7]:
k = 10

### Lazy Sensor Optimisation

In [8]:
sensor_loc_optimisation_lazy(k, sets, X, Y, model_gp)

array([1002,  800,  232, 1064,   29,  964,  536,  500,  972,  263])

### Naive Sensor Optimisation

In [9]:
k = 10
sensor_loc_optimisation_naive(k, sets, X, Y, model_gp)

Exception ignored in: <bound method tqdm.__del__ of   0%|          | 0/10 [04:12<?, ?it/s]>
Traceback (most recent call last):
  File "/Users/adrian/anaconda3/envs/project/lib/python3.6/site-packages/tqdm/_tqdm.py", line 931, in __del__
    self.close()
  File "/Users/adrian/anaconda3/envs/project/lib/python3.6/site-packages/tqdm/_tqdm.py", line 1133, in close
    self._decr_instances(self)
  File "/Users/adrian/anaconda3/envs/project/lib/python3.6/site-packages/tqdm/_tqdm.py", line 496, in _decr_instances
    cls.monitor.exit()
  File "/Users/adrian/anaconda3/envs/project/lib/python3.6/site-packages/tqdm/_monitor.py", line 52, in exit
    self.join()
  File "/Users/adrian/anaconda3/envs/project/lib/python3.6/threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


KeyboardInterrupt: 

In [14]:
k = 100
p_bar = tqdm.tqdm_notebook(range(k))
i = 0
while i < k:
    p_bar.update(1)
    i += 1
    if i == 4 :
        break